In [7]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
from multiprocessing import Process, Pipe
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, SimpleDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

log_dir = './train_log/threshold'
model_loc = './train_log/transfer/all_stages-g2-hidden/all-stages-max-micro-auc.tfmodel'
logger.set_logger_dir(log_dir, action='d')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[1030 21:35:23 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-15effe74-de9e-4182-8f09-718c6089b7bb.json


In [8]:
config = default
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

In [10]:
config.proportion = {'train': 0.8, 'val': 0.2, 'test': 0.0}
config.annotation_number = 10
config.use_hidden_dense = True
config.batch_size = 64
dm = DataManager.from_dataset(train_set, test_set, config)
test_data = dm.get_test_stream()
val_data = dm.get_validation_stream()
dm.get_num_info()

{'test': (3209, 12627), 'train': (2898, 11880), 'val': (678, 2748)}

In [5]:
def run_a_threshold(config, threshold, test_set, pipe):
    model = RNN(config, is_finetuning=False)
    tf.reset_default_graph()
    pred_config = PredictConfig(model=model,
                                session_init=SaverRestore(
                                    model_path=model_loc),
                                output_names=['logits_export', 'label'],
                                )
    # pred = Predictor(pred_config, test_data, nr_proc=2, ordered=False)
    pred = Predictor(pred_config, test_set)

    accumulator = seq(pred.get_result()) \
        .smap(lambda a, b: (a.shape[0], calcu_metrics(a, b, config.validation_metrics, threshold))) \
        .aggregate(Accumulator(*config.validation_metrics), lambda accu, args: accu.feed(args[0], *args[1]))
    metrics = accumulator.retrive()
    pipe.send(metrics)

In [6]:
threshold_choices = [0.4]
metrics = []
for thrsd in threshold_choices:
    receiver, sender = Pipe(duplex=False)
    child = Process(target=run_a_threshold, args=(config, thrsd, val_data, sender))
    child.start()
    child.join()
    metric = receiver.recv()
    print(metric)
    metrics.append(metric)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
[1030 20:57:25 @sessinit.py:89] WRN The following variables are in the graph, but not found in the checkpoint: rnn/process/read_logits/weights, rnn/process/read_logits/biases
[1030 20:57:25 @sessinit.py:89] WRN The following variables are in the checkpoint, but not found in the graph: rnn/process/fc_read/biases:0, rnn/process/fc_read/weights:0
[1030 20:57:30 @s

  8%|8         |1/12[00:04<00:44, 0.25it/s]/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
100%|##########|12/12[00:23<00:00, 0.57it/s]


{'mean_average_precision': 0.67727302818714141, 'macro_auc': 0.94019598495560486, 'micro_auc': 0.94986288468996161, 'macro_f1': 0.55964114972090628, 'micro_f1': 0.6164147167799231, 'ranking_mean_average_precision': 0.74284324807887125, 'coverage': 3.9700520833333335, 'ranking_loss': 0.054404820978869568, 'one_error': 0.30598958333333331}
